In [7]:
from openai import OpenAI
import os
import time

In [ ]:
CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="KEY_GOES_HERE"
)

#model = "meta-llama/llama-3.3-8b-instruct:free"
model = "nvidia/nemotron-nano-12b-v2-vl:free"
# model = "alibaba/tongyi-deepresearch-30b-a3b:free"

In [27]:
# Lettura e riduzione del dataset di partenza
with open("baseline.csv", "r") as f:
    csv_text = f.read()

csv_text_splitted = csv_text.split("\n")

# Anonimizzazione dell'header
old_header = csv_text_splitted[0].split(",")
new_header = ",".join([chr(ord('A') + i) for i in range(len(old_header))])
csv_text_splitted_anon_header = csv_text_splitted
csv_text_splitted_anon_header[0] = csv_text_splitted[0]

# Per contesto mantengo i riferimenti (LETTERA, NOME_VALORE)
ref = {chr(ord('A') + i):old_header[i] for i in range(len(old_header))}

# Prende le prime righe di dati
data = "\n".join(csv_text_splitted[:1000])

In [ ]:
# Interroga il modello
base_prompt = """You are an expert in industrial control systems and in understanding the system architecture by looking at the register values.
In the following, I'll provide you with a list of register values collected over time.

Data:
- This is the scan obtained by observing, for a certain period of time, the contents of the registers of 3 PLCs of an industrial system.
- The first line contains headers that you can use to refer to each register in your answer.

Task:
- The goal is to analyze these values and identify any relationships between columns values.
- Respond clearly with an answer that can be evaluated — in this case, Yes, No or a Number.
"""

q1 = "Q1: Are you able to understand at what type of phisical industrial control system these data refers to?"
q2 = "Q2: Assuming that the industrial system is a water filtration system that certainly uses tanks, are you able to understand how many tanks are involved in the system?"
q3 = "Q3: Are you able to associate the 3 PLCs with the tanks you think you have identified?"
q4 = "Q4: In reality, there are three tanks. Are you able to identify which PLC registers are dedicated to measurements and which to actuations, for each tank?"
q5 = "Q5: In fact, there are three tanks. Are you able to identify which PLC registers are dedicated to measurements and which to actuations, for each tank?"
q6 = "Q6: In fact, there are three tanks. Are you able to identify if there are PLC registers used to store other significant information, such as setpoints or other parameters?"
q7 = "Q7: Considering that there are three tanks, are you able to deduce whether there is a physical connection, i.e., pipes, linking two or more tanks? (This is information that, in our paper, we are not able to derive)"
q8 = "Q8: Assuming you have identified the registers dedicated to containing measurements and actuations for each tank, are you able to construct, for each tank, a chart showing how these quantities, measurements and actuations, evolve over time?"
q9 = "Q9: Are you able to construct a chart representing the operation of the entire system over time?"
q10 = "Q10: For each tank, are you able to extract invariants that relate measurements with actuations?"
q11 = "Q11: Are you able to extract significant invariants that connect the register contents of the entire industrial system?"

questions = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11]
i = 1

context = []


for q in questions:
    c = CLIENT.chat.completions.create(
        model=model,
        messages=[
            {
                "role":"developer",
                "content": [{"type":"text", "text":base_prompt}]
            },
            {
                "role":"assistant",
                "content": [{"type":"text", "text":"\n\n".join(context)},]
            },
            {
                "role":"user",
                "content": [{"type":"text", "text":data}, {"type":"text","text":q}]
            }
        ]
    )
    
    answer = c.choices[0].message.content
    print(f"Question n°{i}\n{answer}\n-------------------------\n\n")

    # Aggiornamento del context
    context.append(answer)

    i+=1

Question n°1
Yes, the data refers to a **multi-PLC (Programmable Logic Controller) industrial control system**, likely used in factory automation, process control, or manufacturing environments. Here's the breakdown:

### Key Observations:
1. **PLC Architecture**:  
   - Terms like `InputRegisters` (e.g., `PLC3_InputRegisters_IW0`), `MemoryRegisters` (e.g., `PLC3_MemoryRegisters_MW1`), and `Coils` (e.g., `PLC1_Coils_QX00`) align with standard PLC memory organization (e.g., Allen Bradley RSLinx/ControlLogix systems).  
   - `Inputs` typically map to sensors (e.g., switches, temperature/level sensors).  
   - `Coils` usually represent outputs controlling actuators (e.g., motors, valves, lights).  
   - `MemoryRegisters` are likely temporary storage for data (e.g., calculated values, flags).

2. **Multiple PLCs**:  
   - The system includes **three PLCs** (PLC1, PLC2, PLC3), suggesting a distributed control architecture where each PLC manages a subsystem (e.g., different machine lines, sa